In [9]:
import swmm.pandas as spd
import swmm_api as api
import swmm.pandas.input.sections as s
import swmm.pandas.input._section_classes as sc
import datetime
import pandas as pd
import numpy.testing as nptest
import numpy as np
from textwrap import dedent
from pprint import pprint

from calendar import month_abbr

def month_to_number(abrr):
    try:
        return list(month_abbr).index(abrr.capitalize())
    except:
        return -1
    
inp = spd.Input('Model.inp')

In [10]:
inp.storage.loc["STOR1", "A1_L"] = 200
inp.storage.loc["STOR1", "desc"] = "shrunk store"



In [12]:
print(';;Name  Elev    MaxDepth  InitDepth  Shape       CurveName  A1_L  A2_W  A0_Z  SurDepth  Fevap  Psi  Ksat  IMD  \n;;----  ------  --------  ---------  ----------  ---------  ----  ----  ----  --------  -----  ---  ----  ---  \nJUNC5   -6.5    13.2      0          TABULAR     Store1                       0         2      2    2     0.5  \n;shrunk store\nSTOR1   -15.25  21.75     0          FUNCTIONAL             200   1     2     10        3                      \n')

;;Name  Elev    MaxDepth  InitDepth  Shape       CurveName  A1_L  A2_W  A0_Z  SurDepth  Fevap  Psi  Ksat  IMD  
;;----  ------  --------  ---------  ----------  ---------  ----  ----  ----  --------  -----  ---  ----  ---  
JUNC5   -6.5    13.2      0          TABULAR     Store1                       0         2      2    2     0.5  
;shrunk store
STOR1   -15.25  21.75     0          FUNCTIONAL             200   1     2     10        3                      



In [11]:
print(inp.storage.to_swmm_string())

;;Name  Elev    MaxDepth  InitDepth  Shape       CurveName  A1_L  A2_W  A0_Z  SurDepth  Fevap  Psi  Ksat  IMD  
;;----  ------  --------  ---------  ----------  ---------  ----  ----  ----  --------  -----  ---  ----  ---  
JUNC5   -6.5    13.2      0          TABULAR     Store1                       0         2      2    2     0.5  
;shrunk store
STOR1   -15.25  21.75     0          FUNCTIONAL             200   1     2     10        3                      



In [19]:
print(inp.outfall.to_swmm_string())

;;Name  Elevation  Type        StageData    Gated  RouteTo  
;;----  ---------  ----------  -----------  -----  -------  
;changed to normal outfall
OUT1    0.1        FREE                     NO              
OUT2    -1.04      FREE                     NO              
OUT3    0.0        TIMESERIES  head_series  YES    SUB1     



In [18]:
        # inp = self.test_base_model
        # self.assertEqual(inp.outfall.reset_index().shape, (3, 7))

        inp.outfall.loc["OUT1", "TYPE"] = "NORMAL"
        inp.outfall.loc["OUT1", "desc"] = "changed to normal outfall"

In [13]:
inp.junc.loc['JUNC4','Elevation']-=5
inp.junc.loc['JUNC4','desc']="dropped invert 5ft"

In [15]:
print(inp.junc.to_swmm_string())

;;Name  Elevation  MaxDepth  InitDepth  SurDepth  Aponded  
;;----  ---------  --------  ---------  --------  -------  
JUNC1   1.5        10.25     0          0         5000     
JUNC2   -1.04      6.2       0          0         5000     
JUNC3   -3.47      11.5      0          0         5000     
;dropped invert 5ft
JUNC4   -10.25     13.8      0          0         5000     
JUNC6   0.0        9.0       0          200       0        



In [2]:
print(inp.timeseries.to_swmm_string())

;;Name           Date       Time       Value     
;;-------------- ---------- ---------- ----------
;SCS_Type_III_1in design storm, total rainfall = 1 in, rain interval = 6 minutes, rain units = in.
SCS_Type_III_1in             0:00 0.00100 
SCS_Type_III_1in             0:06 0.00100 
SCS_Type_III_1in             0:12 0.00100 
SCS_Type_III_1in             0:18 0.00100 
SCS_Type_III_1in             0:24 0.00100 
SCS_Type_III_1in             0:30 0.00100 
SCS_Type_III_1in             0:36 0.00100 
SCS_Type_III_1in             0:42 0.00100 
SCS_Type_III_1in             0:48 0.00100 
SCS_Type_III_1in             0:54 0.00100 
SCS_Type_III_1in             1:00 0.00100 
SCS_Type_III_1in             1:06 0.00100 
SCS_Type_III_1in             1:12 0.00100 
SCS_Type_III_1in             1:18 0.00100 
SCS_Type_III_1in             1:24 0.00100 
SCS_Type_III_1in             1:30 0.00100 
SCS_Type_III_1in             1:36 0.00100 
SCS_Type_III_1in             1:42 0.00100 
SCS_Type_III_1in           

In [1]:
from email import header


def _timeseries_to_swmm_dat(df):
    def time_formatter(x):
        if isinstance(x, pd.Timedelta):
            total_seconds = x.total_seconds()
            hours = int(total_seconds // 3600)  # Get the total hours
            minutes = int((total_seconds % 3600) // 60)  # Get the remaining minutes
            return f"{hours}:{minutes:02}"
        elif isinstance(x, pd.Timestamp):
            return x.strftime("%m/%d/%Y %H:%M")
        elif isinstance(x, (float, int)):
            return x

    def comment_formatter(x):
        if len(x)>0:
            return ";" + x.strip("\n").strip(';').strip().replace("\n", "\n;")
        else:
            return ""

    df = df.reindex(["value", "comment"], axis=1)
    comment = df.attrs.get('comment')
    if len(comment:=df.attrs.get('comment',''))>0:
        comment_line = comment_formatter(comment) + '\n'
    return comment_line+df.reset_index(names="time").to_string(
        formatters=dict(time=time_formatter, comment=comment_formatter),index=False,header=False
    )


In [ ]:
inp.timeseries

In [13]:
inp.timeseries['SCS_Type_III_1in  '].loc['1900-01-01','comment']='this is a commnt\nthis'

In [ ]:
print(_timeseries_to_swmm_dat(inp.timeseries['SCS_Type_III_1in']))

In [ ]:
inp.timeseries

In [3]:
pd.options.display.max_rows = 300

In [17]:
td = pd.Timedelta('50:00:00')

In [ ]:
td.

In [ ]:
def time_formatter(x):
    if isinstance(x,pd.Timedelta):
        total_seconds = x.total_seconds()
        hours = int(total_seconds // 3600)   # Get the total hours
        minutes = int((total_seconds % 3600) // 60)  # Get the remaining minutes
        return f"{hours}:{minutes:02}"
    elif isinstance(x,pd.Timestamp):
        return x.strftime('%m/%d/%Y %H:%M')
    elif isinstance(x,(float,int)):
        return x
print(inp.timeseries['SCS_Type_III_1in']['value'].reset_index(names='time').to_string(formatters=dict(time=time_formatter)))

In [ ]:
inp.timeseries['SCS_Type_III_1in']['value'].reset_index(names='time')

In [ ]:
inp.timeseries['SCS_Type_III_1in'].index.to_series.apply(time_formatter)

In [ ]:
inp.timeseries._timeseries['file_series']

In [ ]:
inp.hydrographs

In [ ]:
print(inp.curves.to_swmm_string())

In [ ]:
inp.curves.to_swmm_string()

In [ ]:
inp.curves.loc[('P1',1),['X_Value','Y_Value']] #= [10,10]

In [ ]:
print(inp.curves)

In [ ]:
print(inp.curves.to_swmm_string())

In [ ]:
print(inp.hydrographs.to_swmm_string())


In [32]:
df = pd.DataFrame(index = [[hyd,rg,''] for hyd,rg in inp.hydrographs.attrs.items()])

In [ ]:
df.index

In [ ]:
[[hyd,rg,''] for hyd,rg in inp.hydrographs.attrs.items()]

In [ ]:
inp.hydrographs.attrs

In [3]:
inp.hydrographs.add_element(Name='Hyd100',Month_RG='RG10',Response='')

In [ ]:
inp.hydrographs

In [ ]:
pd.DataFrame.from_dict(inp.hydrographs.attrs,orient='index')

In [ ]:
def month_to_number(month):
    try:
        return list(month_abbr).index(month.capitalize())
    except ValueError:
        # Return a large number if the month abbreviation is not recognized
        return -1  # This will sort unrecognized months to the end

# Assuming inp.hydrographs is your DataFrame
df = inp.hydrographs

def index_mapper(index):
    if index.name=='Month_RG':
        return index.map(month_to_number)
    else:
        return index
# Sort the DataFrame
sorted_df = df.sort_index().sort_index(ascending=[True,True,False],key=index_mapper)

print(sorted_df)


In [ ]:
inp.hydrographs.index.get_level_values('Month_RG').map(month_to_number)

In [38]:
# Function to safely get level values
def safe_get_level_values(index, level_name):
    try:
        return index.get_level_values(level_name)
    except KeyError:
        return None

# Prepare the sorting key
sort_key = []
for name in ['Name', 'Month_RG', 'Response']:
    level_values = safe_get_level_values(inp.hydrographs.index, name)
    if level_values is not None:
        if name == 'Month_RG':
            sort_key.append(level_values.map(month_to_number))
        else:
            sort_key.append(level_values)


In [ ]:
sort_key

In [ ]:
inp.hydrographs.sort_index().sort_index(key=lambda index: [
    index.get_level_values('Name'),
    index.get_level_values('Month_RG').map(month_to_number),
    index.get_level_values('Response')
])

In [ ]:
inp.hydrographs.iloc[0]

In [ ]:
pd.__version__

In [ ]:
inp.hydrographs.add_element(pd.Series(name='HydrA',index=['Month/RG'],data=[1]))

In [ ]:
inp.hydrographs.__class__.__newobj__

In [ ]:
print(inp.hydrographs.to_swmm_string())

In [20]:
inp = spd.Input('./LID_Model.inp')

In [ ]:
inp.subcatchment.reset_index('Outlet')

In [ ]:
print(inp.lid_usage.to_swmm_string())

In [ ]:
inp.hydrographs

In [ ]:
inp.hydrographs.to_swmm_string()

In [ ]:
print(inp.hydrographs.to_string())

In [12]:
def is_raingauge_row(row):
    non_empty_values = row[row != ''].sum()
    return non_empty_values == 1

def find_raingauge_rows(df):
    # Function to check if a row matches the raingauge criteria

    # Apply the function to each row and get the indices where it's True
    raingauge_indices = df.loc[df.apply(is_raingauge_row, axis=0)].index.tolist()
    
    return raingauge_indices

In [ ]:
inp.hydrographs.iloc[0]

In [ ]:
inp.hydrographs.apply(is_raingauge_row,axis=1)

In [ ]:
find_raingauge_rows(inp.hydrographs)

In [11]:
inp.report.FLOWSTATS

In [ ]:
inp.adjustments

In [13]:
inp.adjustments.loc['TEMPERATURE',:]=0

In [ ]:
print(inp.adjustments.to_swmm_string())

In [ ]:
inp = spd.Input('LID_Model.inp')

In [ ]:
inp.adjustments

In [ ]:
inp.report.LID[1].Fname

In [ ]:
sc.Option.__doc__

In [ ]:
inp.option.__annotations__

In [ ]:
inp.infil._data

In [ ]:
inp.infil.columns

In [ ]:
nptest.assert_equal(
    inp.infil.values,
    np.array([[4.3, 0.86, 0.23, '', '', '', ''],
       [4.3, 0.86, 0.23, '', '', 'MODIFIED_GREEN_AMPT', ''],
       [4.3, 0.86, 0.23, 0.04, 2, 'HORTON', '']],dtype=object)
)

In [ ]:
inp.infil.values

In [ ]:
inp.infil.loc["FAKE_SUB"] = [0,0,0,0,0,"","This is fake"]

In [ ]:
inp.infil.loc['my_really_long_subcatchment_name_that_should','param1'] = 'this'

In [ ]:
print(inp.infil.to_swmm_string())

In [ ]:
inp.subcatchment['Width'] = inp.subcatchment.Area**0.6
# inp.temperature.loc['FILE','param1':'param3'] = ['./climate.dat','1/1/1900','F']

In [ ]:
print(inp.subcatchment.to_swmm_string())

In [ ]:
print(dedent("""\
                ;;Name  RainGage          Outlet  Area  PctImp  Width              Slope  CurbLeng  SnowPack  
                ;;----  ----------------  ------  ----  ------  -----------------  -----  --------  --------  
                SUB1    SCS_Type_III_3in  JUNC1   5     30.83   2.626527804403767  0.5    0         SNOW1     
                SUB2    SCS_Type_III_3in  JUNC2   17    40.74   5.473553316918489  0.5    0         SNOW1     
                SUB3    SCS_Type_III_3in  JUNC4   38    62.21   8.868908165896258  0.5    0         SNOW1     
                """))

In [ ]:
print(inp.adjustments.to_swmm_string().split('\n')[3].strip())

In [ ]:
inp.temperature.loc['FILE','desc']

In [ ]:
inp.temperature.to_swmm_string().split("\n")[7].strip()

In [ ]:
pd.isnull()

In [ ]:
inp.temperature

In [ ]:
inp.temperature.to_swmm_string()

In [ ]:
sc.Temperature.from_section_text(
    """
    TIMESERIES   Temp
    WINDSPEED    MONTHLY    8.900000 11.400000 9.900000 9.800000 8.700000 8.000000 7.300000 7.000000 7.300000 7.800000 8.400000 8.700000
    SNOWMELT               34.000000 0.500000 0.600000 20.000000 41.599998 -4.000000
    ADC          IMPERVIOUS 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
    ADC          PERVIOUS   1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
    """
)

In [ ]:
len(inp.temperature.columns)

In [ ]:
inp.raingage.loc['SCS_Type_III_3in'].values

In [ ]:
inp.option.__class__._from_mgr

In [ ]:
inp.option.loc["ROUTING_STEP", "Value"] = 50
inp.option.loc["ROUTING_STEP", "desc"] = 'Updated routing step'

In [ ]:
inp.option.to_swmm_string().split("\n")[22]

In [ ]:
inp.event.to_swmm_string().split("\n")[4].split()

In [ ]:
inp.option.loc["ROUTING_STEP", "Value"] = 50
inp.option.loc["ROUTING_STEP", "desc"] = 'Updated routing step'

In [ ]:
inp = spd.Input('Model.inp')

In [ ]:
all(inp.raingage.columns == ['Format', 'Interval', 'SCF', 'Source_Type', 'Source', 'Station','Units', 'desc'])

In [ ]:
inp.raingage.iloc[0].values

In [ ]:
inp.raingage.loc['SCS_Type_III_3in','Interval']

In [ ]:
inp.raingage.loc['new_rg'] = ['VOLUME','0:5',1,'FILE','MYFILE','RG1','Inches','my_new_gage']

In [ ]:
print(inp.raingage.to_swmm_string().split('\n')[4])

In [ ]:
len(inp.raingage.columns)

In [ ]:
inp.event.loc[0,'Start']= datetime.datetime(1900,1,2)
inp.event.loc[0,'desc']= 'my first event\nthis is my event'


In [ ]:
inp.event

In [ ]:
print(inp.event.to_swmm_string())

In [ ]:
inp.event.to_swmm_string().split("\n")[3]

In [ ]:
print(inp.event.to_swmm_string().split("\n")[2])

In [ ]:
inp.event

In [ ]:
isinstance(inp.event.Start[0],datetime.datetime)

In [ ]:
print(inp.report.to_swmm_string())

In [ ]:
len(inp.report.to_swmm_string().split('\n')[5].split())

In [ ]:
inp.event

In [ ]:
inp.report.NODES = ['ALL']

In [ ]:
print(inp.report.to_swmm_string().split('\n')[4])

In [ ]:
print(inp.option.to_swmm_string())

In [ ]:
inp.option.loc['START_DATE'] = '01/02/1911'

In [ ]:
inp.report

In [ ]:
inp.title.to_swmm_string()

In [ ]:
print(inp.option.to_swmm_string().split('\n')[-2].strip())

In [ ]:
inp.option.loc["ROUTING_STEP",'Value'] = 50
inp.option.loc["ROUTING_STEP",'desc'] = 'Updated routing step'

In [ ]:
inp.option.to_swmm_string().split("\n")[23].strip()

In [ ]:
inp.option.to_swmm_string().split("\n")

In [ ]:
print(inp.conduit.to_swmm_string())

In [ ]:
print(inp.report.to_swmm_string())

In [ ]:
print(inp.report.to_swmm_string())

In [ ]:
print(inp.event.to_swmm_string())

In [ ]:
from dataclasses import dataclass
from typing import Iterable,Optional

In [ ]:
class Report:
    @dataclass
    class LIDReportEntry:
        Name: str
        Subcatch: str
        Fname: str

    class LIDReport(list):
        def __init__(self, entries: Iterable[Report.LIDReportEntry]):
            for i in entries:
                if not isinstance(i, Report.LIDReportEntry):
                    raise ValueError(
                        f"LIDReport is instantiated with a sequence of LIDReportEntries, got {type(i)}"
                    )
            super().__init__(entries)

        def add(self, lid_name: str, subcatch: str, Fname: str) -> None:
            self.append(
                Report.LIDReportEntry(
                    Name=lid_name,
                    Subcatch=subcatch,
                    Fname=Fname,
                )
            )

        def remove(self, lid_name: str) -> None:
            for i, v in enumerate(self):
                if v.Name == lid_name:
                    break
            self.pop(i)

        def __repr__(self):
            return f"LIDReportList({super().__repr__()})"

    def __init__(
        self,
        disabled: Optional[str] = None,
        input: Optional[str] = None,
        continuity: Optional[str] = None,
        flowstats: Optional[str] = None,
        controls: Optional[str] = None,
        averages: Optional[str] = None,
        subcatchments: list[str] = [],
        nodes: list[str] = [],
        links: list[str] = [],
        lids: list[dict] = [],
    ):
        print(disabled)
        self.DISABLED = disabled
        self.INPUT = input
        self.CONTINUITY = continuity
        self.FLOWSTATS = flowstats
        self.CONTROLS = controls
        self.AVERAGES = averages
        self.SUBCATCH = subcatchments
        self.NODES = nodes
        self.LINKS = links
        self.LIDS = self.LIDReport([])

        for lid in lids:
            self.LIDS.add(lid["name"], lid["subcatch"], lid["fname"])


    @classmethod
    def _from_section_text(cls, text: str, *args, **kwargs) :
        raise NotImplementedError

    @classmethod
    def _new_empty(cls) :
        return cls()

    @classmethod
    def _newobj(cls, *args, **kwargs) :
        return cls(*args,**kwargs)

#


In [ ]:
r = Report()

In [ ]:
r.DISABLED

In [ ]:
t=[1,2,3,4,5,6,7]

In [ ]:
if len(value:=t[1:3])>1:
    print(value)

In [ ]:
t = LIDReport(
    [
        LIDReportEntry('this','that','those'),
        LIDReportEntry('this1','that','those'),
        LIDReportEntry('this2','that','those'),
        LIDReportEntry('this3','that','those')
    ]
)

In [ ]:
t

In [ ]:
t.remove('this')

In [ ]:
t.add(
    'entry1',
    'subcatch2',
    'file'
)

In [ ]:
t

In [ ]:
t = ['this']

In [ ]:
filter()

In [ ]:
t.remove('this')

In [ ]:
t

In [ ]:
inp.event

In [ ]:
inp._section_texts['EVENT']

In [ ]:
input.event

In [ ]:
print(inp.title.to_swmm_string())

In [ ]:
type(super(sc.SectionDf,inp.subarea))

In [ ]:
t = ['list']
t

In [ ]:
t.comment = 'test'

In [ ]:
t.comments

In [ ]:
inp.subarea

In [ ]:
inp.coordinates

In [ ]:
print(inp.xsections.to_swmm_string())

In [ ]:
print(inp._section_texts['STORAGE'])

In [ ]:
print(inp.storage.to_swmm_string())

In [ ]:
inp.re

In [ ]:
inp.infil

In [ ]:
inp.title?

In [ ]:
df = inp.option.copy()

In [ ]:
df.columns=['op','Value','desc']

In [ ]:
inp.option=df

In [ ]:
api_inp = api.SwmmInput('Model.inp')

In [ ]:
api_inp.SUBCATCHMENTS['SUB1']

In [ ]:
inp.subcatchments.loc[0,"CurbLeng"] = 100

In [ ]:
inp.temperature

In [ ]:
inp.temperature.__class__.__name__.lower()

In [ ]:
api_inp.SNOWPACKS